### **ガウス分布の平均,精度パラメタ$\mu,\lambda$の推定**
- 尤度関数(ガウス分布):<br>
$p(x|\mu,\lambda)=\mathcal{N}(x|\mu,\lambda^{-1})$
- 事前分布(ガウス・ガンマ分布):<br>
$p(\mu,\lambda)=\mathcal{N}(x|m,(\beta\lambda_{\mu})^{-1})Gam(\lambda|a,b)$
- 観測データ(ガウス分布から発生):<br>
$\mathcal{D}=\left\{x_1,x_2,x_3,...,x_N\right\},\ x_n\sim\mathcal{N}(x|\mu,\lambda^{-1})+\epsilon,\ \epsilon\sim\mathcal{N}(0,\sigma_{noise})$

In [4]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.animation import ArtistAnimation
from IPython.display import HTML
from tqdm import tqdm
from scipy.stats import norm
from math import gamma
from scipy.special import loggamma

In [41]:
def t_pdf(x,mean,lam,nu):
    A = np.exp(loggamma((nu+1)/2) - loggamma(nu/2))
    B = ((lam/(np.pi*nu))**(1/2))
    C = (1+(lam/nu)*((x-mean)**2))**(-(nu+1)/2)
    
    return A*B*C    


def KLdiv(x,p,q):
    
    width = x[1] - x[0]
    
    p = np.array(p)
    q = np.array(q)
    
    out = np.sum(p*np.log(p/q))*width
    
    return out

In [42]:
# true - gaussian distribution
true_dist = {"mean":5,
             "lambda":1}

#noise
noise_var = 0.5

# prior - gamma distribution
prior = {"mean":1,
         "beta":1,
         "a":1,
         "b":0.001}

In [54]:
#%matplotlib nbagg

artist = []
fig, ax = plt.subplots(2,2,figsize=(10,8))
plt.subplots_adjust(wspace=0.4, hspace=0.6)

frame_num = 40
N_list = 10**np.linspace(0,6,frame_num)

# 左上
ax[0,0].grid()
ax[0,0].set_title("observed data $\mathcal{D}$")
ax[0,0].set_xlabel("$x$",fontsize=15)
Ucb = true_dist["mean"] + 3*true_dist["lambda"]**(-1)
Lcb = true_dist["mean"] - 3*true_dist["lambda"]**(-1)
xlim = ax[0,0].set_xlim(Lcb,Ucb)

# 右上
ax[0,1].grid()
ax[0,1].set_title("posterior $p(\mu,\lambda|\mathcal{D})$")
ax[0,1].set_xlabel("$\mu$",fontsize=15)
ax[0,1].set_ylabel("$\lambda$",fontsize=15)
#ax[0,1].set_xlim([0,1.5])

# 左下
ax[1,0].grid()
ax[1,0].set_title("predictive distribution $\int\int p(x_*|\mu,\lambda)p(\mu,\lambda|\mathcal{D})d\mu d\lambda$")
ax[1,0].set_xlim(xlim)
ax[1,0].set_ylabel("$p(x_*)$",fontsize=15)
ax[1,0].set_xlabel("$x_*$",fontsize=15)

# 右下
ax[1,1].grid()
ax[1,1].set_title("KL-div between true and pred")
ax[1,1].set_xlim(min(N_list),max(N_list))
ax[1,1].set_xlabel("#data",fontsize=15)
ax[1,1].set_xscale("log")
ax[1,1].set_ylabel("KL-divergence $D(p_{true}||p_{pred})$")

est = []
for N in tqdm(N_list):
    
    x = np.linspace(xlim[0],xlim[1],100)
    
    # observe data
    np.random.seed(0)
    data = np.random.normal(loc=true_dist["mean"],
                            scale=true_dist["lambda"]**(-1),
                            size=int(N))
    
    noise = np.random.normal(loc=0,
                             scale=noise_var,
                             size=int(N))
    data += noise
    
    _,_,data_plot = ax[0,0].hist(data,bins=100,color="black")
    
    
    # likelihood
    likelihood = {"mean":true_dist["mean"],
                  "lambda":(data.var())**(-1)}
    
    # posterior - gauss gamma distribution
    posterior = {}
    # gauss
    posterior["beta"] = int(N) + prior["beta"]
    posterior["mean"] = (1/posterior["beta"])*(np.sum(data)+prior["beta"]*prior["mean"])
    # gamma
    posterior["a"] = 0.5*int(N)+prior["a"]
    posterior["b"] = 0.5*(np.sum((data)**2) + prior["beta"]*(prior["mean"]**2) - posterior["beta"]*(posterior["mean"]**2)) + prior["b"]
    
    # visualize
    lam_obs = np.random.gamma(shape=posterior["a"],
                              scale=posterior["b"],
                              size=10000)
    
    posterior_obs = []
    for lam in lam_obs:
        mu_obs = np.random.normal(loc=posterior["mean"],
                                  scale=(posterior["beta"]*lam)**(-1),
                                  size=1)
        sample = [mu_obs[0]]+[lam]
        posterior_obs.append(sample)
    
    posterior_obs = pd.DataFrame(posterior_obs,columns=["mu","lambda"])
    
    H,xedge,yedge = np.histogram2d(x=posterior_obs["mu"].values,
                                   y=posterior_obs["lambda"].values, 
                                   bins=(20,20))
    
    post_plot = ax[0,1].imshow(H.T, interpolation='nearest', origin='lower')
    
    #post_plot = ax[0,1].scatter(x=posterior_obs["mu"].values,
    #                            y=posterior_obs["lambda"].values,
    #                            color="red", marker="x")
    
    
    # predictive - Student t distribution
    pred_dist = {}
    pred_dist["mean"] = posterior["mean"]
    pred_dist["lambda"] = (posterior["beta"]*posterior["a"])/((1+posterior["beta"])*posterior["b"])
    pred_dist["nu"] = 2*posterior["a"]
    
    
    pred = t_pdf(x,pred_dist["mean"],pred_dist["lambda"],pred_dist["nu"])
    true = norm.pdf(x,true_dist["mean"],true_dist["lambda"]**(-1))
    
    hikaku_plot = ax[1,0].plot(x,pred,color="blue",label="predict")
    hikaku_plot += ax[1,0].plot(x,true,color="black",label="true")
    if N == min(N_list):
        ax[1,0].legend()
        #ax[0,1].set_xticks(range(0,20),xedge.astype("str"))
        #ax[0,1].set_yticks(range(0,20),yedge.astype("str"))
    
    # error load
    error = KLdiv(x,true,pred)
    #print(error)
    est.append([N,error])
    est_load = np.array(est)
    error_plot = ax[1,1].plot(est_load[:,0],est_load[:,1],color="black")
    
    frame = data_plot+hikaku_plot+error_plot+[post_plot]
    artist.append(frame)

plt.close()
anim = ArtistAnimation(fig, artist, interval=100)
HTML(anim.to_jshtml())
    

100%|██████████| 40/40 [00:08<00:00,  4.76it/s]


In [44]:
anim.save('gaussian_varmean_estimation_noise05.gif', writer="imagemagick")

MovieWriter imagemagick unavailable; trying to use <class 'matplotlib.animation.PillowWriter'> instead.
